![Picture title](image-20210618-125713.png)

# Machine translation using `simpletransformers` *French to Fongbe and Ewe*

[![simple](https://img.shields.io/badge/Simple_Transformers-v0.61.6-0c0c0c?logo=FutureLearn&logoColor=white&style=for-the-badge)](https://github.com/ThilinaRajapakse/simpletransformers)

This library is based on the Transformers library by HuggingFace. Simple Transformers lets you quickly train and evaluate Transformer models. Only 3 lines of code are needed to initialize a model, train the model, and evaluate a model.

**Supports**

- Sequence Classification
- Token Classification (NER)
- Question Answering
- Language Model Fine-Tuning
- Language Model Training
- Language Generation
- T5 Model
- Seq2Seq Tasks
- Multi-Modal Classification
- Conversational AI.
- Text Representation Generation.

[![Picture title](image-20210601-095719.png)](https://zindi.africa/competitions/ai4d-takwimu-lab-machine-translation-challenge/leaderboard)

The objective of this challenge is to create a machine translation system capable of converting text from French into Fongbe or Ewe. You may train one model per language or create a single model for both. You may not use any external data, so a key component of this competition is finding a way to work with the available data efficiently.

# installing simple transformers

In [ ]:
!pip install simpletransformers
!pip install fsspec==2021.5.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data
I have used only 35k samples so that my GPU dosent run out of memory and I have used original data without preprocessing.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/drive/MyDrive/Fon_and_ewe/Train.csv")[0:35000]# Change Train Data directory
test = pd.read_csv("/content/drive/MyDrive/Fon_and_ewe/Test.csv")# Change Test Data Directory

In [ ]:
Clean = False


Cleaning function was use full in english translation but in this case the we get poor results if we clean the data.

In [ ]:
if Clean:    
    # converting every letter to lower case
    df['Target'] = df['Target'].apply(lambda x: str(x).lower())
    df['French'] = df['French'].apply(lambda x: str(x).lower())
    # removing apostrophe from the sentences
    df['Target'] = df['Target'].apply(lambda x: re.sub("'","",x))
    df['French'] = df['French'].apply(lambda x: re.sub("'","",x))
    exclude = set(string.punctuation)
    # removing all the punctuations
    df['Target'] = df['Target'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
    df['French'] = df['French'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
    # removing digits from the sentences
    digit = str.maketrans('','',string.digits)
    df['Target'] = df['Target'].apply(lambda x: x.translate(digit))
    df['French'] = df['French'].apply(lambda x: x.translate(digit))

Dividing train and test data into two dataframe based on languagues.

In [ ]:
Fon = df[df.Target_Language=="Fon"]
Ewe = df[df.Target_Language=="Ewe"]
Fon_test = test[test.Target_Language=="Fon"]
Ewe_test = test[test.Target_Language=="Ewe"]

# Training `Fon` Model
Using simple transformer seq2seq I have downloaded `Helsinki-NLP/opus-mt-en-mul` which work best in our case and using specific `Seq2SeqArgs` to set arguments of model.

**Args**
- num_train_epochs = 30
- batch_size = 32
- max_length = 120
- src_lang ="fr"
- tgt_lang ="fon"
- overwrite_output_dir = True


In [ ]:
import logging

import pandas as pd
from simpletransformers.seq2seq import (
    Seq2SeqModel,
    Seq2SeqArgs,
)


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_data = Fon[["French","Target"]]

train_data = train_data.rename(columns={"French":"input_text","Target":"target_text"})
train_df, eval_df = train_test_split(train_data, test_size=0.2, random_state=42)


model_args = Seq2SeqArgs()
model_args.num_train_epochs = 30
model_args.no_save = True
model_args.evaluate_generated_text = False
model_args.evaluate_during_training = False
model_args.evaluate_during_training_verbose = True
model_args.rag_embed_batch_size = 32
model_args.max_length = 120
model_args.src_lang ="fr"
model_args.tgt_lang ="fon"
model_args.overwrite_output_dir = True

# Initialize model
model_fon = Seq2SeqModel(
    encoder_decoder_type="marian",
    encoder_decoder_name="Helsinki-NLP/opus-mt-en-mul",
    args=model_args,
    use_cuda=True,
)


def count_matches(labels, preds):
    print(labels)
    print(preds)
    return sum(
        [
            1 if label == pred else 0
            for label, pred in zip(labels, preds)
        ]
    )







INFO:filelock:Lock 140299461074704 acquired on /root/.cache/huggingface/transformers/1f25261d7e9a5b5e6ce297ed0729be5ee5cfcf28a0036e3f80eba1960bb4b2d9.d51a6de0226a95f56f1f97bb6b6aefc907cc894dd36c443cdd34f429f486314c.lock


Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

INFO:filelock:Lock 140299461074704 released on /root/.cache/huggingface/transformers/1f25261d7e9a5b5e6ce297ed0729be5ee5cfcf28a0036e3f80eba1960bb4b2d9.d51a6de0226a95f56f1f97bb6b6aefc907cc894dd36c443cdd34f429f486314c.lock
INFO:filelock:Lock 140299461074064 acquired on /root/.cache/huggingface/transformers/8650f5a471109bb8de32913dacd2bee19092b32c39c393cfb30675f482886691.62b69e0eee5ff18e923501b8a53937012624d329597babff911a60dd241f839b.lock


Downloading:   0%|          | 0.00/310M [00:00<?, ?B/s]

INFO:filelock:Lock 140299461074064 released on /root/.cache/huggingface/transformers/8650f5a471109bb8de32913dacd2bee19092b32c39c393cfb30675f482886691.62b69e0eee5ff18e923501b8a53937012624d329597babff911a60dd241f839b.lock
INFO:filelock:Lock 140299314459536 acquired on /root/.cache/huggingface/transformers/a7a6c8809d4a0131388179669009ceb4a9bcb87e315164edc72d88c1e5cb6f64.9d4a3fe1aecf1f27f4bb2a123653e1b92a085d6da1f2487b963804345bf9f1a5.lock


...

INFO:filelock:Lock 140299307294416 released on /root/.cache/huggingface/transformers/93cb3e0edab5f3c9ea3fbf684629f2e91ac08d2b634d4c4c4907639aa562275f.3303786501bbf15c469ef0b5b3589c297c4da13961186d5df23ac58f3f3d815e.lock
INFO:filelock:Lock 140299307359312 acquired on /root/.cache/huggingface/transformers/6c54153c8cf71106c9acc6066ac37e607ffa9fd6de37553173b43d0218ba55ed.9c2c44ec310a412d87b8c543b37ddff3a0a4c28520077d5278349b38ca545899.lock


Downloading:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

INFO:filelock:Lock 140299307359312 released on /root/.cache/huggingface/transformers/6c54153c8cf71106c9acc6066ac37e607ffa9fd6de37553173b43d0218ba55ed.9c2c44ec310a412d87b8c543b37ddff3a0a4c28520077d5278349b38ca545899.lock
INFO:filelock:Lock 140299307359312 acquired on /root/.cache/huggingface/transformers/3b817f217dcfaca02a6b7c4ab26d74a815ea8d85bce59fc546d5a709ae66a581.24f4e214d77dcd35586715bf2de661069b19eb8abc7696ada0af21ddf8a65e45.lock


Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

INFO:filelock:Lock 140299307359312 released on /root/.cache/huggingface/transformers/3b817f217dcfaca02a6b7c4ab26d74a815ea8d85bce59fc546d5a709ae66a581.24f4e214d77dcd35586715bf2de661069b19eb8abc7696ada0af21ddf8a65e45.lock


## Training Model

In [ ]:
# Train the model
model_fon.train_model(
    train_df, eval_data=eval_df, matches=count_matches
)


INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/19738 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model: Training started


Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Running Epoch 0 of 30:   0%|          | 0/2468 [00:00<?, ?it/s]

...

Running Epoch 28 of 30:   0%|          | 0/2468 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-70000
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-71572-epoch-29


Running Epoch 29 of 30:   0%|          | 0/2468 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-72000
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-74000
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-74040-epoch-30
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/
INFO:simpletransformers.seq2seq.seq2seq_model: Training of Helsinki-NLP/opus-mt-en-mul model complete. Saved to outputs/.


(74040, 0.6869546678079066)

## Model Evaluation

In [ ]:
# # Evaluate the model
results = model_fon.eval_model(eval_df)

In [ ]:
results

## Sample Prediction

In [ ]:
# Use the model for prediction
print(
    model_fon.predict(
        
            Fon_test["French"].values[25]
        
    )
)

Generating outputs:   0%|          | 0/24 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3260: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


['Dɔnkpá', 'ɖokpó sɔ́', 'élɔ́ mɛ̀', 'xojlawemajlakpà', 'kɔkɔ́kɔ́ɖɔ́', 'gɔn ɛ́', 'ɖagbe', 'yi atín', 'gbɔ̀n gbɔ̀n', 'ɖokpo', 'hălù', 'ɔ́, alo wŭ', 'asì - asɔ ɔ́', 'nyì lanmὲ', 'ɖù nù nùsáwŭ', 'nu e', 'Ffrika wá', 'é ɔ́, ɖò hŭn yì', 'wàn', 'ɖo ɛ̀', 'ɖĕbŭ', 'wè', 'é nyɔ́ nú mì', 'ɖe jijɔ']


## Predicting `FONGBE` Test Data

In [ ]:
Fon_test["Target"] = model_fon.predict(list(Fon_test["French"].values))

Generating outputs:   0%|          | 0/367 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3260: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Saving csv file

In [ ]:
Fon_test[["ID","Target"]].to_csv("Fon.csv",index=False)

## Saving the model

In [ ]:
import torch

In [ ]:
torch.save(model_fon , 'model_fon.pkl')

# Training `EWE` Model
Using simple transformer seq2seq I have downloaded `Helsinki-NLP/opus-mt-en-mul` which work best in our case and using specific `Seq2SeqArgs` to set arguments of model.

**Args**
- num_train_epochs = 30
- batch_size = 32
- max_length = 120
- src_lang ="fr"
- tgt_lang ="ewe"
- overwrite_output_dir = True

In [ ]:
import logging

import pandas as pd
from simpletransformers.seq2seq import (
    Seq2SeqModel,
    Seq2SeqArgs,
)


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
train_data = Ewe[["French","Target"]]

train_data = train_data.rename(columns={"French":"input_text","Target":"target_text"})
train_df, eval_df = train_test_split(train_data, test_size=0.20, random_state=42)


model_args = Seq2SeqArgs()
model_args.num_train_epochs = 30
model_args.no_save = True
model_args.evaluate_generated_text = False
model_args.evaluate_during_training = False
model_args.evaluate_during_training_verbose = True
model_args.rag_embed_batch_size = 32
model_args.max_length = 120
model_args.src_lang ="fr"
model_args.tgt_lang ="ewe"
model_args.overwrite_output_dir = True
# Initialize model
model_ewe = Seq2SeqModel(
    encoder_decoder_type="marian",
    encoder_decoder_name="Helsinki-NLP/opus-mt-en-mul",
    args=model_args,
    use_cuda=True,
)


In [ ]:

def count_matches(labels, preds):
    print(labels)
    print(preds)
    return sum(
        [
            1 if label == pred else 0
            for label, pred in zip(labels, preds)
        ]
    )

## Training Model

In [ ]:
model_ewe.train_model(
    train_df, eval_data=eval_df, matches=count_matches
)

INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/8261 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model: Training started


Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Running Epoch 0 of 30:   0%|          | 0/1033 [00:00<?, ?it/s]

...

Running Epoch 28 of 30:   0%|          | 0/1033 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-29957-epoch-29


Running Epoch 29 of 30:   0%|          | 0/1033 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-30000
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-30990-epoch-30
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/
INFO:simpletransformers.seq2seq.seq2seq_model: Training of Helsinki-NLP/opus-mt-en-mul model complete. Saved to outputs/.


(30990, 1.3801951767043092)

## Predicting on `EWE` Test Dataset

In [ ]:
Ewe_test["Target"]=model_ewe.predict(list(Ewe_test["French"].values))

Generating outputs:   0%|          | 0/371 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3260: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Saving into csv file

In [ ]:
Ewe_test[["ID","Target"]].to_csv("Ewe.csv",index=False)

## Saving Model

In [ ]:
torch.save(model_ewe , 'model_ewe.pkl')

# Submission
concating both predicted data sets

In [ ]:
ewe = pd.read_csv('Ewe.csv')
fon = pd.read_csv('Fon.csv')
fr_to_targ_lang_sub = pd.concat([ewe, fon])
fr_to_targ_lang_sub.head()

,ID,Target
0,ID_AAAAhgRX,eƒe vɔvɔ̃ hã
1,ID_AChdWHyF,nyigba gbɔgbɔ
2,ID_AHBSoUNL,ele abe esia ene... » » Basa
3,ID_AHycIkQv,xlẽa nu vɔ̃ sɔgbɔ
4,ID_AIWTdKBT,"ɖeka be akpɔ akplasi kple adzreɖeɖeame ɖe ta, ..."


## Submission file `FON and EWE`

In [ ]:
fr_to_targ_lang_sub.to_csv('/content/drive/MyDrive/Fon_and_ewe/submission1.csv', index=False)

# Results

![Picture title](image-20210618-124722.png)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e019ab02-6596-4ab8-a99f-fb27d5d7445f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>